# <div style="padding: 30px; color:white; margin:10; font-size:95%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b> Intel Review - Web Scraping - Amazon Scraper using BeautifulSoup</b></div>

<div style="background-color: #3b3745; border-radius: 12px; padding: 20px; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);">
    <h2 style="color: #F1A424; text-align: center;">Table of Contents</h2>
    <ul style="list-style: none; padding: 0;">
        <li><a href="#section-1" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">1.</span>  Importing Libraries</a></li>
        <li><a href="#section-2" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">2.</span>Reading Urls from Text File</a></li>
        <li><a href="#section-3" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">3.</span>Fetching rendered HTML content from URLs </a></li>
        <li><a href="#section-4" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">4.</span> Scraping User Reviews from Website</a></li>
        <li><a href="#section-5" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">5.</span> Iterating through domains and pages, fetching reviews</a></li>
        <li><a href="#section-6" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">6.</span>  Initial Data Pre-Processing</a></li>
        <li><a href="#section-7" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">7.</span>  Saving Reviews to CSV</a></li> 
    </ul>
</div>

<a id="section-1"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>1 |</span></b> <b>  Importing Libraries</b></div>

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from langdetect import detect, DetectorFactory

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<a id="section-2"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>2 |</span></b> <b> Reading Urls from Text File</b></div>

In [38]:
filename = 'amazon_in_urls.txt'
file = open('amazon_in_urls.txt')
LIST=re.split("\r\n|\n",file.read())
file.close()
base_url = LIST[0].split('~')[-1]
product_name = str(LIST[1])
filename.split(".")

['amazon_in_urls', 'txt']

In [39]:
reviewlist = []
domains = [".com", ".co.uk", ".ca", ".in", ".co.mx", ".com.au", ".co.jp", ".sg", ".nl", ".de", ".fr", ".it", ".es", ".com.br"]


<a id="section-3"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>3 |</span></b> <b> Fetching rendered HTML content from URLs</b></div>

In [40]:
def get_soup(url):
    r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

<a id="section-4"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>4 |</span></b> <b> Scraping User Reviews from Website</b></div>

In [41]:
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            product_title = soup.title.text.strip()
            product_title = product_title.split("Intel", 1)[-1].strip()
            product_title = "Intel " + product_title

            review_date = item.find('span', {'data-hook': 'review-date'}).text.strip()
            review_date = ' '.join(review_date.split()[-3:])

            rating_text = item.find('i', {'data-hook': 'review-star-rating'}).text.strip()
            rating_value = re.search(r'(\d+(\.\d+)?)', rating_text).group(1)
            rating = float(rating_value)

            review = {
                'Product': product_title,
                'Rating': rating,
                'Date': review_date,
                'Review': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(review)
    except Exception as e:
        print(f"An error occurred: {e}")

<a id="section-5"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>5 |</span></b> <b> Iterating through domains and pages, fetching reviews</b></div>

In [42]:
for y in domains:
    for x in range(1,11):
        url = re.sub(r'www\.amazon\.[a-z\.]+', f'www.amazon{y}', base_url)
        url = f'{url}&pageNumber={x}'
        soup = get_soup(url)
        print(f'Getting page: {x} from {y}')
        initial_length = len(reviewlist)
        get_reviews(soup)
        print(len(reviewlist))
        print(url)
        if len(reviewlist) == initial_length:
            break
        if not soup.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break

Getting page: 1 from .com
10
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
Getting page: 2 from .com
20
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
Getting page: 3 from .com
30
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
Getting page: 4 from .com
40
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
Getting page: 5 from .com
50
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
Getting page: 6 from

<a id="section-6"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>6 |</span></b> <b> Initial Data Pre-Processing</b></div>

In [ ]:
DetectorFactory.seed = 0

custom_words = [
    "against", "above", "below", "out", "under", "again",
    "no", "nor", "not", "don't", "should", "didn't",
    "does", "doesn't", "hadn't", "hasn't", "haven't",
    "isn't", "mustn", "mustn't", "should", "shouldn't",
    "was", "wasn't", "weren", "weren't", "won't", "wouldn't",
    "very", "couldn't"
]

stop_words = set(stopwords.words('english'))
for word in custom_words:
    if word in stop_words:
        stop_words.remove(word)

lemmatizer = WordNetLemmatizer()

for i in range(len(reviewlist)):
    review_dict = reviewlist[i]
    review = review_dict['Review']
    
    # Detect the language of the review
    try:
        language = detect(review)
        if language == 'ja':  # Skip Japanese reviews
            print(f"Skipping Japanese review: {review}")
            continue
    except Exception as e:
        print(f"Error detecting language: {e}")
        continue
    
    # Remove non-alphabetic characters
    review = re.sub(r'[^A-Za-z\s]', '', review)
    
    # Convert to lowercase
    review = review.lower()
    
    # Tokenize and remove stopwords
    words = word_tokenize(review)
    filtered_words = [word for word in words if word not in stop_words]
    
    # Lemmatize
    lemmatized_review = ' '.join([lemmatizer.lemmatize(word) for word in filtered_words])
    
    review_dict['Review'] = lemmatized_review


<a id="section-7"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>7 |</span></b> <b> Saving Reviews to CSV</b></div>

In [44]:
df = pd.DataFrame(reviewlist)
df.to_csv(f'../data/amazon/{product_name}.csv', index=False)
print('Fin.')

Fin.


                                   This concludes the analysis presented in this notebook.
